In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pygwalker as pyg

### Read the data into a dataframe

In [2]:
# read the data

df = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name=None, header=1)

In [3]:
print(df.keys())

dict_keys(['Title Sheet', 'Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress'])


###  Assigning sheets to dataframe objects

In [4]:
transactions_df = df['Transactions']
New_customers_df = df['NewCustomerList']
customer_demographic_df = df['CustomerDemographic']
customer_address_df = df['CustomerAddress']
title_docx = df['Title Sheet']

## Assess the datasets one after the other:

#### Transaction dataset assessment

In [9]:
transactions_df.sample(10)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
6950,6951,73,3242,2017-11-11,0.0,Approved,Solex,Standard,medium,medium,1945.43,333.18,41848.0
8163,8164,74,30,2017-12-19,1.0,Approved,WeareA2B,Standard,medium,medium,1228.07,400.91,33429.0
9035,9036,81,818,2017-10-03,1.0,Approved,Norco Bicycles,Standard,medium,small,586.45,521.94,33429.0
18063,18064,78,1041,2017-12-10,0.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,38193.0
9135,9136,41,598,2017-04-06,0.0,Approved,Norco Bicycles,Standard,low,medium,958.74,748.90,38693.0
18113,18114,95,647,2017-06-25,0.0,Approved,Giant Bicycles,Standard,medium,large,569.56,528.43,37337.0
18234,18235,0,1674,2017-09-27,0.0,Approved,OHM Cycles,Road,high,large,12.01,7.21,39880.0
10191,10192,19,1370,2017-11-27,1.0,Approved,OHM Cycles,Road,high,large,12.01,7.21,39880.0
4145,4146,3,3268,2017-05-09,0.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41009.0
15894,15895,66,264,2017-01-26,1.0,Approved,Giant Bicycles,Road,low,small,590.26,525.33,40487.0


In [10]:
transactions_df.shape

(20000, 13)

There are 20000 entries in the dataset.

### Note that, we're going to catch the issues as we explore the dataset which will be documented at the bottom of this notebook.

In [11]:
# first check for null values

transactions_df.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

there are null values in the dataset, which indicates an issue of completeness.

In [22]:
# check for duplicate transactions

transactions_df.duplicated().all() == False

True

No duplicate transactions.

In [23]:
transactions_df.product_first_sold_date.sample(5)

11367    42295.0
17738    34556.0
11489    39526.0
13796    42688.0
19217    42458.0
Name: product_first_sold_date, dtype: float64

Entries in product_first_sold_date column have to be of date data type but they're float. This raises an issues of validity

In [24]:
transactions_df.online_order.value_counts()

1.0    9829
0.0    9811
Name: online_order, dtype: int64

online_order column must be boolean data type but we have float. Floating points do not give a relevant meaning to the column name online_order hence it raises an issue of validity.

In [25]:
transactions_df.order_status.value_counts()

Approved     19821
Cancelled      179
Name: order_status, dtype: int64

In [26]:
transactions_df.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date'],
      dtype='object')

In [27]:
transactions_df.transaction_date.sample(10)

7554    2017-10-10
4808    2017-07-18
15401   2017-05-22
5780    2017-04-09
4564    2017-08-26
6153    2017-06-24
16905   2017-08-15
10411   2017-10-06
7650    2017-08-01
6595    2017-01-27
Name: transaction_date, dtype: datetime64[ns]

In [37]:
# check the range(max-min) in the transaction_date column to see if its <=3.

transactions_df.transaction_date.max() - transactions_df.transaction_date.min()

Timedelta('363 days 00:00:00')

As shown in the above cell, the range of transaction_date is not 3 months. The dataset consist of about one year of transaction data which needs to be reduced to 3 months by choosing a reference date.

In [38]:
# check customer_id and transaction_id for duplicate entries.


transactions_df.customer_id.duplicated().any()

True

There is an indication of duplicate entries in customer_id because a single customer can perform multiple transactions or can purchase many products from the shop.

In [66]:
transactions_df[['customer_id', 'transaction_id']].duplicated().any()

False

In [63]:
transactions_df[transactions_df.duplicated(subset=['customer_id', 'transaction_id'])]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date


In [67]:
transactions_df[['transaction_id']].duplicated().any()

False

There are no duplicated transactions in the transactions dataset.

### New customers dataset assessment

In [68]:
customers_df = New_customers_df

In [69]:
customers_df.shape

(1000, 23)

There are 1000 customers in the customers dataset

In [70]:
customers_df.columns

Index(['first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'address', 'postcode', 'state', 'country',
       'property_valuation', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Rank', 'Value'],
      dtype='object')

In [72]:
customers_df.sample(20)

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
748,Mikol,Eck,Male,99,1975-07-25,VP Marketing,Financial Services,Mass Customer,N,Yes,...,QLD,Australia,9,0.90,1.1250,1.406250,1.195312,748,748,0.650781
837,Monty,Thomazin,Male,7,1951-09-16,Quality Engineer,NaN,Mass Customer,N,Yes,...,VIC,Australia,10,0.79,0.9875,1.234375,1.049219,838,838,0.573750
60,Lorrie,Antonelli,Male,47,1983-02-11,Cost Accountant,Financial Services,High Net Worth,N,Yes,...,NSW,Australia,7,0.40,0.5000,0.500000,0.500000,57,57,1.375000
887,Adriena,Giffin,Female,27,1957-03-17,Analog Circuit Design manager,Argiculture,Affluent Customer,N,Yes,...,NSW,Australia,9,0.84,1.0500,1.312500,1.312500,888,888,0.525000
470,Ceciley,Harg,Female,50,1990-10-19,Nurse Practicioner,Retail,Mass Customer,N,Yes,...,QLD,Australia,7,0.49,0.6125,0.612500,0.520625,471,471,0.887500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,Farra,Matyushkin,Female,18,1974-01-24,VP Quality Control,Manufacturing,High Net Worth,N,Yes,...,VIC,Australia,9,0.79,0.9875,1.234375,1.234375,241,241,1.087500
763,Simmonds,Bapty,Male,52,2002-01-04,Junior Executive,Financial Services,Affluent Customer,N,No,...,NSW,Australia,12,0.46,0.4600,0.575000,0.575000,760,760,0.637500
435,Skipp,Swales,Male,15,1973-11-14,Community Outreach Specialist,Entertainment,High Net Worth,N,Yes,...,VIC,Australia,7,0.91,1.1375,1.137500,1.137500,436,436,0.903125
286,Taber,Szymon,Male,70,1947-04-22,Senior Sales Associate,Argiculture,Affluent Customer,N,No,...,QLD,Australia,8,1.03,1.0300,1.030000,1.030000,287,287,1.040000


In [73]:
customers_df.isnull().sum()

first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Unnamed: 16                              0
Unnamed: 17                              0
Unnamed: 18                              0
Unnamed: 19                              0
Unnamed: 20                              0
Rank                                     0
Value                                    0
dtype: int6

There are null values in the last_name, DOB, job_title and job_industry_category

In [78]:
customers_df.duplicated().all()

False

In [ ]:
customers_df.

There are duplicates in the customers dataset.

In [79]:
customers_df.owns_car.value_counts()

No     507
Yes    493
Name: owns_car, dtype: int64

In [80]:

customer_address_df.sample(20)

,customer_id,address,postcode,state,country,property_valuation
3658,3663,954 Reinke Park,3101,VIC,Australia,12
3987,3992,8 Randy Parkway,2209,NSW,Australia,11
2742,2747,5 Lakewood Gardens Court,2118,NSW,Australia,9
2344,2349,99 Eastlawn Way,2360,NSW,Australia,1
2479,2484,0 Union Plaza,2444,NSW,Australia,7
...,...,...,...,...,...,...
992,997,1 Continental Center,2146,NSW,Australia,9
993,998,0465 Melrose Alley,2768,NSW,Australia,9
3359,3364,62551 Del Mar Avenue,2300,NSW,Australia,6
1925,1930,3542 Arkansas Place,3380,VIC,Australia,1


In [81]:
customer_address_df.shape

(3999, 6)

In [82]:
customer_address_df.address.nunique()

3996

In [96]:
# investigate postcode and address..

customer_address_df.duplicated(subset=['address', 'postcode']).any()

False

No duplicate address and postcode

In [86]:
customer_demographic_df.sample(10)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
2312,2313,Christa,Pauley,Female,54,1966-07-10,Chemical Engineer,Manufacturing,Mass Customer,N,Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ,No,12.0
970,971,Sarge,Lerohan,Male,11,1988-10-25,Geologist I,Manufacturing,High Net Worth,N,ã»(ï¿£âï¿£)ã»:*:,No,22.0
601,602,Lebbie,Bruck,Female,0,1992-03-28,Office Assistant II,NaN,High Net Worth,N,åè£½æ¼¢èª,Yes,9.0
3523,3524,Waiter,Piscopello,Male,38,1973-12-30,Design Engineer,Financial Services,Mass Customer,N,1,Yes,18.0
1494,1495,Charmion,Dobsons,Female,4,1989-08-14,Physical Therapy Assistant,Health,High Net Worth,N,ã,Yes,8.0
1688,1689,Teodor,Larwell,Male,48,1995-06-18,Quality Control Specialist,Financial Services,Mass Customer,N,â¡,Yes,4.0
365,366,Constance,Ucchino,Female,7,1954-01-08,Librarian,Entertainment,High Net Worth,N,â©testâ©,No,19.0
523,524,Florence,Held,Female,16,1962-09-09,Nurse,Retail,Mass Customer,N,ï¾ï½¥â¿ã¾â²(ï½¡ââ¿âï½¡)â±â¿ï½¥ï¾,No,11.0
2805,2806,Thorndike,Glossop,Male,60,1986-05-18,Programmer Analyst II,Manufacturing,Mass Customer,N,NaN,Yes,3.0
915,916,Joycelin,NaN,Female,18,1991-06-18,Recruiter,NaN,Affluent Customer,N,TÌÌ­ÌºÌºoÍ Ì·iÌ²Ì¬ÍÌªÍnÌÌÍvÍÌÌÌÌ¦oÌ¶ÌÌ°Ì keÍÍÌ®ÌºÌªÌ¹Ì±Ì¤ ÌtÍÌÍÌ³Ì£Ì»ÌªhÌ¼ÍÌ²Ì¦Ì³ÌÌ²eÍÌ£Ì°Ì¦Ì¬Í Ì¢Ì¼Ì»Ì±ÌhÍÍÍÍÌÌ£Ì²iÌ¦Ì²Ì£Ì°Ì¤vÌ»ÍeÌºÌ­Ì³ÌªÌ°-mÌ¢iÍnÌÌºÌÌ²Ì¯Ì°dÌµÌ¼ÌÍÌ©Ì¼ÌÌ³ ÌÌ¥Ì±Ì³Ì­rÌÌÌeÍpÍ rÌ¼ÌÌ»Ì­ÌeÍÌºÌ Ì£sÌ,No,8.0


In [85]:
customer_demographic_df.deceased_indicator.value_counts()

N    3998
Y       2
Name: deceased_indicator, dtype: int64

### Data quality Issues:
> Completenes:
    >1. There are null values in the online_order, brand, product_line, product_class, `product_size`, `standard_cost`, `product_first_sold_date` columns.
    >2. In customers dataset, last_name, DOB, `job_title` and `job_industry_category` contain null values.
    >3. 
    
> Consistency:
    >1. `List_price` and `standard_cost` columns of some products are not consistent. We can have two products of the same size, class, brand and product_line with different `list_price` and different `standard_cost`. 
    
> Accuracy:
    >1. In customer demographic dataset, the `gender` column has an incorrect value `U`, which could represent a customer who didn't give information about their gender. That it not accurate.
    >2. `default` column in customer demographic contains inaccurate values. some of which are symbols and foreign/unknown characters.
    >3. in customer demographic, tenure does not have a clear column name. It needs to be changed to a clearer and concise name
    
> Currency:
    >1. `Transaction_date` column contains data points that are not current,i.e., they are over three(3) months old. This can be resolved by getting the last three months of transaction data.
    >2. Customers dataset contains information about customers' purchases in the last three (3) years which is also irrelevant since we're considering the last three (3) months of data.
    
> Relevancy:
    >1. `Product_class` column is not relevant since there is also product_size columns. Both seem to have same format and interpretation therefore it is irrelevant to make use of both.
    >2. `product_first_sold_date` column is not relevant in a transaction database.
    >3. Customers dataset contains information about customers' purchases in the last three (3) years which is also irrelevant.
    >4. `deceased_indicator` in customers' dataset is not relevant.
    >5. In the customers' dataset, `Rank` and `Values` and other `unnamed columns` have no meaning in the dataset. 
    
> Validity:
    >1. `product_first_sold_date` cloumn's data points are not valid date/datetime values.
    >2. `online_order` has invalid data types
    
> Uniquesness:
    >1. There exist duplicate entries in the customers' dataset.
    >2. 
    
